In [ ]:
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_div_std = grp_CAGRUI_mean / grp_CAGRUI_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_div_std = grp_CAGRStd_mean / grp_CAGRStd_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_div_std = grp_StdUI_mean / grp_StdUI_std

  return df_perf, grp_StdUI_mean_div_std, grp_CAGRStd_mean_div_std, grp_CAGRUI_mean_div_std